In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gcmotion as gcm

# for debugging, can be removed
from pprint import pprint as pp
gcm.utils.reload()

plt.style.use("ggplot")
plt.rcParams["figure.dpi"] = 150

### Variables and Equations

The purpose of this tool is to calculate the orbit of different particles in a toroidal magnetic field $\vec B$ with large aspect ratio (LAR) and constant $g,I$. The orbits are calculated by solving Hamiltons equations as presented in "The theory of toroidally confined plasmas" by R. B. White,with a 4th order Runge-Kutta method

Our dynamical variables are  $\theta, P_\theta, \zeta, P_\zeta$ and $ \rho_{||}$, and they are governed through Hamilton's equations:

$$
\dot \theta = \dfrac{1}{D}\rho_{||}B^2 +\dfrac{g}{D}\bigg[ -(\mu+\rho^2_{||}B)\bigg(\dfrac{q}{\sqrt{2\psi}}\cos(\theta)\bigg)  + \dfrac{\partial \Phi}{\partial \psi_p}\bigg]
$$

$$
\dot \psi_p = \dfrac{\dot\psi}{q(\psi)} = -\dfrac{g}{D}\bigg[(\mu + \rho_{||}^2B)\sqrt{2\psi}\sin(\theta) + \dfrac{\partial \Phi}{\partial \theta}\bigg]
$$

$$
\dot \rho_{||} = -\dfrac{1}{D}\bigg[(\mu + \rho_{||}^2B)\sqrt{2\psi}\sin(\theta) + \dfrac{\partial \Phi}{\partial \theta} \bigg]
$$

$$
\dot \zeta = \dfrac{\rho_{||}B^2}{D} - \dfrac{I}{D}\bigg[ -(\mu+\rho^2_{||}B)\bigg(\dfrac{q}{\sqrt{2\psi}}\cos(\theta)\bigg)  + \dfrac{\partial \Phi}{\partial \psi_p}\bigg]
$$

where:  $\quad B = 1 - \sqrt{2\psi}\cos(\theta)$

We can then calculate the canonical momenta as $P_\theta = \psi + \rho_{||}I$ and $P_\zeta = \rho_{||} g -\psi_p$

### Part 1: Particle Initialization and obrit calculation

In this cell we can create multiple particles independently. By setting cwp (current working particle), we chose the particle to proceed. That way we can keep all previous particles in memory and switch cwp at any point.

First we set up the Tokamak configuration, and the particle's initial conditions

In [ ]:
R, a = 6.2, 2# Major Radius in [m]
q = gcm.qfactor.Hypergeometric(R, a)
Bfield = [0, 1, 5] # I, g, B0 (Field strength on axis in [T])
Efield = gcm.efield.Radial(R, a, q, Ea = 7500, minimum = 0.95, waist_width=50)

# Create Particle
species = "p"
mu = 10**(-5) # Magnetic moment
theta0 = 0
psi0 = 0.75# times psi_wall
z0 = 0
Pz0 = -0.035
tspan = np.linspace(0, 500000, 100000) # t0, tf, steps

init_cond = [theta0, psi0, z0, Pz0]

#Particle Creation
Electron1 = gcm.Particle(species, mu, init_cond, tspan, R, a, q, Bfield, Efield)
cwp = Electron1
cwp.run()

### Part 2: Plots

In [ ]:
cwp.plot.tokamak_profile()

In [ ]:
cwp.plot.time_evolution(percentage =100)

### Energy Contour Plots

In [ ]:
cwp.plot.contour_energy(
    theta_lim = [-np.pi ,np.pi], psi_lim = "auto", 
    plot_drift=True, contour_Phi = True, units = "keV", levels = 20
)

## Orbit types parabolas

### Parabolas Derivation

The two left parabolas are defined in White as:

$$
\dfrac{(P_\zeta + \psi_{wall})^2B^2_{min}}{2g^2(\psi_{wall})}+\mu B_{min}+\Phi_{wall}-E=0
$$

We can re-order the terms as such:

$$
\bigg(\dfrac{\mu B_0}{E} \bigg) = - \dfrac{B_{min}B_0\psi^2_{wall}}{2g^2E}
\bigg(\dfrac{P_\zeta}{\psi_{wall}}\bigg)^2 - 
\dfrac{B_{min}B_0\psi^2_{wall}}{g^2E}\bigg(\dfrac{P_\zeta}{\psi_{wall}}\bigg)-
\dfrac{B_{min}B_0\psi^2_{wall}}{2g^2E} + \dfrac{B_0}{B_{min}}
\bigg(1-\dfrac{\Phi_{wall}}{E}\bigg) 
$$

where $E = \mu B_{min}+\Phi_{wall}$

***

Similarly, for the right parabola, we have:

$$
\bigg(\dfrac{\mu B_0}{E} \bigg) = - \dfrac{B_0^2\psi^2_{wall}}{2g^2E}\bigg(\dfrac{P_\zeta}{\psi_{wall}}\bigg)^2 + 1 - \dfrac{\Phi_0}{E}
$$

where $E = \mu B_{0}+\Phi_{0}$

***

As for the Trapped-Passing Boundary:

$$
\bigg(\dfrac{\mu B_0}{E} \bigg) = \dfrac{1}{1\pm\sqrt{-2\psi_{wall}\bigg(\dfrac{P_\zeta}{\psi_{wall}}\bigg)}}\bigg(1 - \dfrac{\Phi_0}{E}\bigg)
$$

***

By defining the new variables:

$$
x=\bigg(\dfrac{P_\zeta}{\psi_{wall}}\bigg)\quad,\quad y = \bigg(\dfrac{\mu B_0}{E} \bigg)
$$

we define a parabola $y = \alpha x^2 + \beta x + \gamma$. Now we can easily find the minima/maxima and axis intercepts.

Note: Since $B = 1 - \sqrt{2\psi}\cos\theta \quad$,   $\psi_{wall}$ should be less that 0.5

As for figuring out if the particle is Lost or Confined, we need only to check if its orbit point lies in-between the 2 left parabolas.

### Parabolas Diagram

In [ ]:
cwp.plot.parabolas()
cwp.plot.orbit_point()

## Torus Poincare map and 3d Torus orbits

In [ ]:
cwp.plot.torus2d(percentage = 100, truescale=True)

In [ ]:
cwp.plot.torus3d(percentage=100, truescale=True, 
                white_background=True, hd=False,)

In [ ]:
# params = {"percentage": 100, "truescale": True,
#           "min_step": 0.01, "seconds": 300}
# gcm.animate(cwp, params)

## FFT

In [ ]:
from scipy import fft
from scipy.signal import argrelextrema as ex

x = cwp.z
span = ex(x, np.greater)[0][:2]
x1 = x[span[0] : span[1]]
plt.plot(tspan[:x1.shape[0]], x1)

dt = tspan[1]-tspan[0]
f = 1/tspan[x1.shape[0]]
sample_rate = 1/dt
print(f"Frequency from graph: f = {f}")
print(f"Sample rate = {sample_rate}")

In [ ]:
X = fft.rfft(x1)/x1.shape[0] # Divide to sample size
freq = fft.rfftfreq(x1.shape[0])*sample_rate


fig, ax = plt.subplots(figsize = (10,5))
markerline, stemlines, baseline = ax.stem(
    freq, np.abs(X), linefmt = "blue")
markerline.set_markersize(4)
stemlines.set_linewidths(.8)
ax.set_xlim(np.array([-sample_rate/10, sample_rate])/100);
